In [ ]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel


In [ ]:
df = pd.read_csv("your_dataset.csv")


In [ ]:
X_train, X_test, y_train, y_test, category_train, category_test = train_test_split(
    df["X"], df["label"], df["category"], test_size=0.2, random_state=42
)


In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def preprocess_text(text):
    encoded_text = tokenizer(text, return_tensors="pt", padding=True)
    return encoded_text["input_ids"].to(device), encoded_text["attention_mask"].to(device)


In [ ]:
from transformers import BertTokenizer, BertModel

# Hyperparameters
num_classes = 3  # Number of dark pattern categories
embedding_dim = 768  # Dimension of BERT embeddings
dropout_rate = 0.2

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased")

# Define device (CPU or GPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
class ResNetTransformer(nn.Module):
    def __init__(self, embedding_dim, num_classes, dropout_rate):
        super(ResNetTransformer, self).__init__()
        self.bert = bert_model

        # Define residual connections
        self.layer1 = nn.Sequential(
            nn.Linear(embedding_dim, embedding_dim // 2),
            nn.ReLU(),
            nn.Linear(embedding_dim // 2, embedding_dim),
            nn.Dropout(dropout_rate),
        )
        self.layer2 = nn.Sequential(
            nn.Linear(embedding_dim, embedding_dim // 2),
            nn.ReLU(),
            nn.Linear(embedding_dim // 2, embedding_dim),
            nn.Dropout(dropout_rate),
        )

        # Classification layer
        self.fc = nn.Linear(embedding_dim, num_classes)

    def forward(self, input_ids, attention_mask):
        bert_output = self.bert(input_ids, attention_mask=attention_mask)
        hidden_states = bert_output[0]

        # Residual connections
        residual = hidden_states
        hidden_states = self.layer1(hidden_states)
        hidden_states += residual
        residual = hidden_states
        hidden_states = self.layer2(hidden_states)
        hidden_states += residual

        # Classification
        logits = self.fc(hidden_states[:, 0, :])  # Take CLS token representation

        return logits

# Create model instance
model = ResNetTransformer(embedding_dim, num_classes, dropout_rate).to(device)


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = nn.CrossEntropyLoss()


In [ ]:
for epoch in range(num_epochs):
    train_loss = 0
    model.train()  # Set model to training mode

    for i in range(0, len(X_train), batch_size):
        batch_text = X_train[i:i+batch_size]
        batch_labels = y_train[i:i+batch_size]

        input_ids, attention_mask = preprocess_text(batch_text)

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        logits = model(input_ids, attention_mask)
        loss = loss_fn(logits, batch_labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Print training loss after each epoch
    train_loss = train_loss / len(X_train)
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.4f}")


In [ ]:
with torch.no_grad():
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0

    for i in range(0, len(X_test), batch_size):
        batch_text = X_test[i:i+batch_size]
        batch_labels = y_test[i:i+batch_size]

        input_ids, attention_mask = preprocess_text(batch_text)

        # Forward pass
        logits = model(input_ids, attention_mask)
        predictions = torch.argmax(logits, dim=1)

        # Calculate accuracy
        correct += (predictions == batch_labels).sum().item()
        total += batch_labels.size(0)

    accuracy = correct / total
    print("Validation accuracy:", accuracy)
